# 0.**Librerias**


 A lanzar en L4

In [ ]:
!pip install -U torch --index-url https://download.pytorch.org/whl/cu121
!pip install -U transformers accelerate datasets bitsandbytes triton peft trl \
    bert-score textstat numpy pandas
# ===============================
# ✅ LIMPIEZA DE SALIDA Y VERIFICACIÓN DE VERSIONES
# ===============================
import IPython
IPython.display.clear_output(wait=True)

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 118.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 91.0 

In [ ]:
import torch, transformers, peft, trl, accelerate, bitsandbytes, datasets

print("✅ Librerías cargadas correctamente:\n")
print(f"PyTorch:        {torch.__version__}")
print(f"Transformers:   {transformers.__version__}")
print(f"PEFT:           {peft.__version__}")
print(f"TRL:            {trl.__version__}")
print(f"Accelerate:     {accelerate.__version__}")
print(f"Datasets:       {datasets.__version__}")
print(f"BitsAndBytes:   {bitsandbytes.__version__}")

print("\n✨ Entorno listo para QLoRA 4-bit + SFTTrainer + TRL 0.24.0 ✨")

✅ Librerías cargadas correctamente:

PyTorch:        2.9.0+cu126
Transformers:   4.57.1
PEFT:           0.18.0
TRL:            0.25.1
Accelerate:     1.12.0
Datasets:       4.4.1
BitsAndBytes:   0.48.2

✨ Entorno listo para QLoRA 4-bit + SFTTrainer + TRL 0.24.0 ✨


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Carga de la Colección Cochrane

In [ ]:
# 1. Crear carpeta y entrar
!mkdir cochrane_data
%cd cochrane_data

# 2. Inicializar git vacío
!git init

# 3. Agregar remote
!git remote add -f origin https://github.com/feliperussi/bridging-the-gap-in-health-literacy.git

# 4. Activar modo sparse checkout
!git config core.sparseCheckout true

# 5. Indicar SOLO las carpetas que queremos
!echo "data_collection_and_processing/Data Sources/Cochrane/train/pls" >> .git/info/sparse-checkout
!echo "data_collection_and_processing/Data Sources/Cochrane/train/non_pls" >> .git/info/sparse-checkout

# 6. Descargar solo esa parte del repo
!git pull origin main


/content/cochrane_data
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/cochrane_data/.git/
Updating origin
remote: Enumerating objects: 72074, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 72074 (delta 0), reused 2 (delta 0), pack-reused 72071 (from 2)
Receiving objects: 100% (72074/72074), 315.90 MiB | 15.92 MiB/s, done.
Resolving deltas: 100% (2991/2991), done.
From https://github.com/feliperussi/bridging-the-gap-in-health-literacy
 * [new branch]

Gestionar Hugging Face

In [ ]:
import os
os.environ["HF_TOKEN"] = "hf_xxx"

In [ ]:
from huggingface_hub import login
login(token=os.environ["HF_TOKEN"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Librerias

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model
import glob, pandas as pd
import transformers, trl, peft, accelerate, datasets
import bitsandbytes as bnb
import re

# 1.Construir el dataset non_pls → pls desde las carpetas de GitHub

In [ ]:
# Número máximo de pares
pares = 2000

# === 1️⃣ Listar archivos ===
pls_files = sorted(glob.glob("data_collection_and_processing/Data Sources/Cochrane/train/pls/*.txt"))
non_pls_files = sorted(glob.glob("data_collection_and_processing/Data Sources/Cochrane/train/non_pls/*.txt"))

# === 2️⃣ Filtrar archivos que contengan 'accumulated' en AMBOS ---
pls_files = [f for f in pls_files if "accumulated" not in os.path.basename(f).lower()]
non_pls_files = [f for f in non_pls_files if "accumulated" not in os.path.basename(f).lower()]

print(f"📚 Total PLS (sin accumulated): {len(pls_files)}")
print(f"📂 Total non-PLS (sin accumulated): {len(non_pls_files)}")

# === 3️⃣ Función para extraer el ID base ===
def extract_id(filename):
    base = os.path.basename(filename)
    match = re.match(r"(.+?)\.pub\d+", base)
    return match.group(1) if match else base.split(".txt")[0]

# === 4️⃣ Diccionarios {id_base: ruta} ===
pls_dict = {extract_id(p): p for p in pls_files}
non_pls_dict = {extract_id(n): n for n in non_pls_files}

# === 5️⃣ Emparejar ===
data = []
for base_id, non_path in non_pls_dict.items():
    if base_id in pls_dict:
        pls_path = pls_dict[base_id]
        with open(pls_path, "r", encoding="utf-8") as f1, open(non_path, "r", encoding="utf-8") as f2:
            data.append({
                "id": base_id,
                "file_pls": os.path.basename(pls_path),
                "file_non_pls": os.path.basename(non_path),
                "pls": f1.read().strip(),
                "non_pls": f2.read().strip()
            })
    if len(data) >= pares:
        break

# === 6️⃣ DataFrame ===
df = pd.DataFrame(data)

# === 7️⃣ Guardar ===
output_path = "cochrane_pairs_clean.csv"
df.to_csv(output_path, index=False, encoding="utf-8")

print(f"\n✅ Emparejados {len(df)} pares por nombre base (sin 'accumulated' en ninguno)")
print(f"💾 Guardado en: {output_path}")
df.head(3)



📚 Total PLS (sin accumulated): 4797
📂 Total non-PLS (sin accumulated): 7251

✅ Emparejados 2000 pares por nombre base (sin 'accumulated' en ninguno)
💾 Guardado en: cochrane_pairs_clean.csv


,id,file_pls,file_non_pls,pls,non_pls
0,10.1002-14651858.CD000006,10.1002-14651858.CD000006.pub2-pls.txt,10.1002-14651858.CD000006.pub2-abstract.txt,Absorbable stitches for repair of episiotomy a...,Background\nApproximately 70% of women will ex...
1,10.1002-14651858.CD000009,10.1002-14651858.CD000009.pub4-pls.txt,10.1002-14651858.CD000009.pub4-abstract.txt_se...,Do acupuncture and related therapies help smok...,We included 38 studies. Based on three studies...
2,10.1002-14651858.CD000012,10.1002-14651858.CD000012.pub4-pls.txt,10.1002-14651858.CD000012.pub4-abstract.txt_se...,Alternative versus conventional institutional ...,"Ten trials involving 11,795 women met the incl..."


# 2.Test,train split y generacion del prompt

In [ ]:
df = df.rename(columns={"non_pls": "source", "pls": "target"})

INSTR_PROMPT = """Using the following abstract of a biomedical study as input, generate a Plain Language Summary
(PLS) understandable by any patient, regardless of their health literacy. Ensure that the generated text
adheres to the following instructions which should be followed step-by-step:
a. Specific Structure: The generated PLS should be presented in a logical order, using the following
order:
1. Plain Title
2. Rationale
3. Trial Design
4. Results
b. Sections should be authored following these parameters:
1. Plain Title: Simplified title understandable to a layperson that summarizes the research that was
done.
2. Rationale: Include: background or study rationale providing a general description of the
condition, what it may cause or why it is a burden for the patients; the reason and main hypothesis
for the study; and why the study is needed, and why the study medication has the potential to
treat the condition.
3. Trial Design: Answer ‘How is this study designed?’ Include the description of the design,
description of study and patient population (age, health condition, gender), and the expected
amount of time a person will be in the study.
4. Results: Answer ‘What were the main results of the study’, include the benefits for the patients,
how the study was relevant for the area of study, and the conclusions from the investigator.
c. Consistency and Replicability: The generated PLS should be consistent regardless of the order of
sentences or the specific phrasing used in the input protocol text.
d. Compliance with Plain Language Guidelines: The generated PLS must follow all these plain
language guidelines:
• Have readability grade level of 6 or below.
• Do not have jargon. All technical or medical words or terms should be defined or broken down
into simple and logical explanations.
• Active voice, not passive.
• Mostly one or two syllable words.
• Sentences of 15 words or less.
• Short paragraphs of 3-5 sentences.
• Simple numbers (e.g., ratios, no percentages).
e. Do not invent Content: The AI model should not invent information. If the AI model includes data
other than the one given in the input abstract, the AI model should guarantee such data is verified and
real.
f. Aim for an approximate PLS length of 500-900 words.

Input abstract:

{source}

Output PLS:
"""

def format_example(row):
    prompt = INSTR_PROMPT.format(source=row["source"])
    return {"prompt": prompt, "response": row["target"]}

# Aplicar la función a cada fila del DataFrame
dataset = df.apply(format_example, axis=1).to_list()

# Crear Dataset de Hugging Face y dividir en train/test
from datasets import Dataset

hf_ds = Dataset.from_list(dataset).train_test_split(test_size=0.1, seed=42)

print(hf_ds)


DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 1800
    })
    test: Dataset({
        features: ['prompt', 'response'],
        num_rows: 200
    })
})


## Mostrar uno al azar

In [ ]:
import random

idx = random.randint(0, len(hf_ds["train"]) - 1)
print(f"🔹 Ejemplo #{idx}")
print(hf_ds["train"][idx]["prompt"])

🔹 Ejemplo #160
Using the following abstract of a biomedical study as input, generate a Plain Language Summary
(PLS) understandable by any patient, regardless of their health literacy. Ensure that the generated text
adheres to the following instructions which should be followed step-by-step:
a. Specific Structure: The generated PLS should be presented in a logical order, using the following
order:
1. Plain Title
2. Rationale
3. Trial Design
4. Results
b. Sections should be authored following these parameters:
1. Plain Title: Simplified title understandable to a layperson that summarizes the research that was
done.
2. Rationale: Include: background or study rationale providing a general description of the
condition, what it may cause or why it is a burden for the patients; the reason and main hypothesis
for the study; and why the study is needed, and why the study medication has the potential to
treat the condition.
3. Trial Design: Answer ‘How is this study designed?’ Include the descri

# 3.Training del Modelo (Modelo Base + QLoRA)

version Llama

In [ ]:
MODELO="meta-llama/Llama-3.2-3B-Instruct"
num_train_epochs=4 ### Aquí se indica el numero de Epoch
COSINE_WEIGHT=0  #0.05
KL_WEIGHT=0.1   #0.05
READABILITY_WEIGHT=0
COMPLEXITY_WEIGHT=0
base_model_name="meta-llama/Llama-3.2-3B-Instruct" #teacher para KL

## Cargar modelo anterior

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForSeq2Seq, BitsAndBytesConfig, AutoModel,TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
import torch
import torch.nn as nn
import torch.nn.functional as F
import gc
import os
import sys
import torch
from torch import nn
from transformers import Trainer
import textstat
import pyphen
import re


print("\n" + "="*80)
print("🚀 SETUP - OPCIÓN C FIXED")
print("="*80)

# 1. Cargar modelo base
model_id = MODELO
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

print(f"\n📥 Cargando {model_id}...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


#=============================================================================
 #Cargar embedding model:
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)
embedding_model = AutoModel.from_pretrained(embedding_model_name).to("cpu")


# ============================================================================
# CUSTOM LOSS CON COSENO
# ============================================================================

# ============================================================================
# NUEVA VERSIÓN DIFERENCIABLE DEL COSINE LOSS (usa hidden states del modelo)
# ============================================================================

import torch
import torch.nn as nn
import torch.nn.functional as F

"""
============================================================================
OPCIÓN C - FIXED: Proyector registrado en el modelo
============================================================================

PROBLEMA IDENTIFICADO:
- El proyector tenía gradientes pero NO estaba en el optimizer
- Grupo 1 del optimizer: 0 params

SOLUCIÓN:
- Crear proyector ANTES del trainer
- Registrarlo como model.cosine_proj
- Pasarlo al loss
- El Trainer lo optimizará automáticamente

============================================================================
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoModel,
    Trainer,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import gc

# ============================================================================
# PASO 1: Loss Function (recibe proyector externo)
# ============================================================================

class CosineAlignedLoss(nn.Module):
    """
    Loss combinada: CrossEntropy + Cosine + KL (regularización lingüística)
    """
    def __init__(self, embedding_model=None, embedding_tokenizer=None,
                 cosine_weight=0, kl_weight=0,
                 readability_weight=0,   # 🆕 AÑADIR
                 complexity_weight=0,    # 🆕 AÑADIR
                 proj=None, ref_model=None):
        super().__init__()
        self.embedding_model = embedding_model
        self.embedding_tokenizer = embedding_tokenizer
        self.proj = proj
        self.ref_model = ref_model  # modelo base congelado (sin gradientes)
        self.cosine_weight = cosine_weight
        self.kl_weight = kl_weight

        self.ce_loss = nn.CrossEntropyLoss(ignore_index=-100, reduction="mean") ###<--
        self._debug_count = 0
        # NUEVOS atributos
        self.readability_weight = 0
        self.complexity_weight = 0
        self.pyphen_dic = pyphen.Pyphen(lang='en')
        self.step_counter = 0
        self.compute_text_metrics_every = 5  # Calcular cada 5 steps
        self.cached_readability = 0.0
        self.cached_complexity = 0.0

    # =============================================================
# PASO 2: Funciones auxiliares (añadir a tu clase)
# =============================================================

    def _compute_readability_loss(self, texts, device):
      """Calcula Flesch-Kincaid Grade Loss"""
      if not texts:
          return torch.tensor(0.0, device=device, requires_grad=True)  # 🔸 Añadir

      loss = torch.tensor(0.0, device=device, requires_grad=True)  # 🔸 Añadir
      valid_count = 0

      for text in texts:
          text = text.strip()
          if not text or len(text) < 10:
              continue

          try:
              grade = textstat.flesch_kincaid_grade(text)

              if grade > 6.0:
                  penalty = ((grade - 6.0) / 6.0) ** 2
                  # 🔸 CAMBIO IMPORTANTE: Crear tensor con requires_grad
                  penalty_tensor = torch.tensor(
                      penalty,
                      device=device,
                      dtype=torch.float32,
                      requires_grad=True  # 🔸 AÑADIR
                  )
                  loss = loss + penalty_tensor

              valid_count += 1

          except Exception:
              loss = loss + torch.tensor(1.0, device=device, requires_grad=True)  # 🔸 Añadir
              valid_count += 1

      return loss / valid_count if valid_count > 0 else loss


    def _compute_complexity_loss(self, texts, device):
      """Calcula Complexity Loss"""
      if not texts:
          return torch.tensor(0.0, device=device, requires_grad=True)  # 🔸 Añadir

      loss = torch.tensor(0.0, device=device, requires_grad=True)  # 🔸 Añadir
      valid_count = 0

      for text in texts:
          text = text.strip()
          if not text or len(text) < 10:
              continue

          try:
              # Frases largas
              sentences = re.split(r'[.!?]+', text)
              sentence_penalty = 0.0

              for sentence in sentences:
                  words = sentence.strip().split()
                  if len(words) > 15:
                      sentence_penalty += (len(words) - 15) / 15

              # Palabras complejas
              all_words = re.findall(r'\b[a-zA-Z]+\b', text.lower())

              if all_words:
                  complex_count = sum(
                      1 for w in all_words
                      if len(self.pyphen_dic.inserted(w).split('-')) >= 3
                  )

                  complex_ratio = complex_count / len(all_words)
                  word_penalty = (complex_ratio - 0.10) * 2.0 if complex_ratio > 0.10 else 0.0
              else:
                  word_penalty = 0.0

              total_penalty = sentence_penalty + word_penalty
              # 🔸 CAMBIO: Crear tensor con requires_grad
              penalty_tensor = torch.tensor(
                  total_penalty,
                  device=device,
                  dtype=torch.float32,
                  requires_grad=True  # 🔸 AÑADIR
              )
              loss = loss + penalty_tensor
              valid_count += 1

          except Exception:
              loss = loss + torch.tensor(0.5, device=device, requires_grad=True)  # 🔸 Añadir
              valid_count += 1

      return loss / valid_count if valid_count > 0 else loss

    def forward(self, outputs, labels, texts, inputs=None,tokenizer=None):
      """
      Calcula la pérdida total = CE + (cosine_weight * Cosine) + (kl_weight * KL)
      Compatible con entrenamiento puro CE o combinado (coseno / distillation).
      """

      # =============================================================
      # 🔹 CROSS-ENTROPY (con shift)
      # =============================================================
      shift_logits = outputs.logits[..., :-1, :].contiguous()
      shift_labels = labels[..., 1:].contiguous()

      ce_loss = self.ce_loss(
          shift_logits.view(-1, shift_logits.size(-1)),
          shift_labels.view(-1)
      )

      total_loss = ce_loss
      cosine_loss = torch.tensor(0.0, device=outputs.logits.device)
      kl_loss = torch.tensor(0.0, device=outputs.logits.device)

      # NUEVOS: Inicializar readability y complexity
      readability_loss = torch.tensor(0.0, device=outputs.logits.device, requires_grad=True)
      complexity_loss = torch.tensor(0.0, device=outputs.logits.device, requires_grad=True)

      # =============================================================
      # 🔹 COSINE LOSS (si está activado)
      # =============================================================
      if (
          self.cosine_weight > 0
          and self.embedding_model is not None
          and self.proj is not None
          and hasattr(outputs, "hidden_states")
          and outputs.hidden_states is not None
          and texts  # 👈 evita listas vacías
      ):
          hidden = outputs.hidden_states[-1]
          device = hidden.device

          with torch.no_grad():  # solo congelamos el embedding model
              emb_inputs = self.embedding_tokenizer(
                  texts, padding=True, truncation=True, max_length=512,
                  return_tensors="pt"
              )
              emb_out = self.embedding_model(**emb_inputs.to("cpu"))

          ref_emb = emb_out.last_hidden_state.mean(dim=1)
          ref_emb = F.normalize(ref_emb, dim=-1).to(device).detach()

          # Promediar las hidden states del modelo student (solo tokens válidos)
          output_mask = (labels != -100).float().unsqueeze(-1)
          output_hidden = (hidden * output_mask).sum(1) / (output_mask.sum(1) + 1e-8)
          output_emb = F.normalize(self.proj(output_hidden), dim=-1)

          cosine_sim = F.cosine_similarity(output_emb, ref_emb, dim=-1)
          if torch.isnan(cosine_sim).any():
              cosine_sim = torch.nan_to_num(cosine_sim, nan=0.0)
          cosine_loss = (1 - cosine_sim).mean()

          total_loss = total_loss + self.cosine_weight * cosine_loss
      # =============================================================
      # 🔹 KL DIVERGENCE (teacher-student distillation)
      # =============================================================

      if self.kl_weight > 0 and self.ref_model is not None and inputs is not None:

          ref_device = next(self.ref_model.parameters()).device
          student_device = outputs.logits.device

          # --- Forward del teacher congelado ---
          with torch.no_grad():
              ref_inputs = {
                  k: v.to(ref_device)
                  for k, v in inputs.items()
                  if isinstance(v, torch.Tensor)
              }

              ref_logits = self.ref_model(**ref_inputs, return_dict=True).logits

              # Mover SIEMPRE al device del student
              ref_logits = ref_logits.to(student_device)

          # --- Forward del student ---
          stud_logits = outputs.logits

          # --- Alinear longitudes ---
          min_len = min(stud_logits.size(1), ref_logits.size(1))
          stud_logits = stud_logits[:, :min_len, :]
          ref_logits  = ref_logits[:, :min_len, :]

          # --- KL Divergence (temp=2) ---
          p = F.log_softmax(stud_logits / 2.0, dim=-1)
          q = F.softmax(ref_logits / 2.0, dim=-1)
          kl_loss = F.kl_div(p, q, reduction="batchmean") * 4.0

          total_loss = total_loss + self.kl_weight * kl_loss

      # =============================================================
      # 🔹 READABILITY LOSS (NUEVO)
      # =============================================================
      if self.readability_weight > 0 and texts and tokenizer is not None:
          self.step_counter += 1

          # Calcular solo cada N steps para velocidad
          if self.step_counter % self.compute_text_metrics_every == 0:

              # Generar texto si no está disponible
              # (texts puede ser referencias, necesitamos generar del modelo)
              with torch.no_grad():
                  generated_ids = outputs.logits.argmax(dim=-1)
                  generated_texts = tokenizer.batch_decode(
                      generated_ids,
                      skip_special_tokens=True
                  )

              # Calcular readability
              readability_loss = self._compute_readability_loss(
                  generated_texts,
                  device=outputs.logits.device
              )

              # Cachear para próximos steps
              self.cached_readability = readability_loss.item()

          else:
              # Usar valor cacheado (pequeña aproximación)
              readability_loss = torch.tensor(
                  self.cached_readability,
                  device=outputs.logits.device,
                  dtype=torch.float32,
                  requires_grad=True  # 🔸 AÑADIR

              )

          total_loss = total_loss + self.readability_weight * readability_loss

      # =============================================================
      # 🔹 COMPLEXITY LOSS (NUEVO)
      # =============================================================
      if self.complexity_weight > 0 and texts and tokenizer is not None:

          # Calcular junto con readability (mismo paso)
          if self.step_counter % self.compute_text_metrics_every == 0:

              # Usar mismos generated_texts de arriba
              with torch.no_grad():
                  if 'generated_texts' not in locals():
                      generated_ids = outputs.logits.argmax(dim=-1)
                      generated_texts = tokenizer.batch_decode(
                          generated_ids,
                          skip_special_tokens=True
                      )

              # Calcular complexity
              complexity_loss = self._compute_complexity_loss(
                  generated_texts,
                  device=outputs.logits.device
              )

              # Cachear
              self.cached_complexity = complexity_loss.item()

          else:
              # Usar cacheado
              complexity_loss = torch.tensor(
                  self.cached_complexity,
                  device=outputs.logits.device,
                  dtype=torch.float32,
                  requires_grad=True  # 🔸 AÑADIR
              )

          total_loss = total_loss + self.complexity_weight * complexity_loss


      # =============================================================
      # 🧠 Debug (primeras iteraciones)
      # ============================================================

      if self._debug_count < 3:
        print(f"🔍 CE={ce_loss.item():.4f}")
        print(f"🔹 Cos={cosine_loss.item():.4f} (×{self.cosine_weight} = +{(self.cosine_weight*cosine_loss).item():.4f})")
        print(f"🔹 KL={kl_loss.item():.4f} (×{self.kl_weight} = +{(self.kl_weight*kl_loss).item():.4f})")
        print(f"🔹 READ={readability_loss.item():.4f} (×{self.readability_weight} = +{(self.readability_weight*readability_loss).item():.4f})")
        print(f"🔹 COMP={complexity_loss.item():.4f} (×{self.complexity_weight} = +{(self.complexity_weight*complexity_loss).item():.4f})")
        print(f"✅ Total={total_loss.item():.4f}\n")
        self._debug_count += 1

        print(f"grad CE={ce_loss.requires_grad}, Cos={cosine_loss.requires_grad}, KL={kl_loss.requires_grad}, READ={readability_loss.requires_grad}, COMP={complexity_loss.requires_grad}, Total={total_loss.requires_grad}")

      return total_loss, ce_loss, cosine_loss, kl_loss, readability_loss, complexity_loss



# ============================================================================
# ESTADÍSTICAS
# ============================================================================

stats_train = {
    "prompt_tokens_original": [],
    "prompt_tokens_used": [],
    "prompt_start_kept": [],
    "prompt_end_kept": [],
    "prompt_middle_removed": [],
    "response_tokens_original": [],
    "response_tokens_used": [],
    "both_fit_count": 0,
    "prompt_truncated_count": 0,
    "prompt_truncated_middle_count": 0,
    "response_truncated_count": 0,
}

# ============================================================================
# PREPROCESAMIENTO
# ============================================================================

def preprocess_function(examples, max_length=1536, min_response_preserve=0.95, ###1536
                        start_ratio=0.35, collect_stats=False):
    MIN_RESPONSE_TOKENS = 50

    model_inputs = {"input_ids": [], "attention_mask": [], "labels": [], "source_texts": []}

    for prompt, response in zip(examples["prompt"], examples["response"]):
        if not response or len(response.strip()) == 0:
            continue

        # Extraer source text del prompt (entre ### Original text: y ### Simplified text:)
        if "Input abstract:" in prompt and "Output PLS:" in prompt:
            source_text = prompt.split("Input abstract:")[1].split("Output PLS:")[0].strip()
        else:
            source_text = prompt

        prompt_tokens = tokenizer(prompt, add_special_tokens=False)
        response_tokens = tokenizer(response, add_special_tokens=False)

        prompt_ids = prompt_tokens["input_ids"]
        response_ids = response_tokens["input_ids"]

        original_prompt_len = len(prompt_ids)
        original_response_len = len(response_ids)

        if original_response_len < MIN_RESPONSE_TOKENS:
            continue

        if collect_stats:
            stats_train["prompt_tokens_original"].append(original_prompt_len)
            stats_train["response_tokens_original"].append(original_response_len)

        prompt_was_truncated = False
        prompt_was_truncated_middle = False
        response_was_truncated = False

        available_for_response = max_length - MIN_RESPONSE_TOKENS

        if original_response_len <= available_for_response:
            final_response_ids = response_ids
            available_for_prompt = max_length - len(final_response_ids)
        else:
            min_response_tokens = max(
                int(original_response_len * min_response_preserve),
                MIN_RESPONSE_TOKENS
            )
            if min_response_tokens > max_length * 0.6:
                min_response_tokens = int(max_length * 0.6)

            final_response_ids = response_ids[:min_response_tokens]
            response_was_truncated = True
            available_for_prompt = max_length - len(final_response_ids)

        if original_prompt_len <= available_for_prompt:
            final_prompt_ids = prompt_ids
            if collect_stats:
                stats_train["both_fit_count"] += 1
                stats_train["prompt_start_kept"].append(original_prompt_len)
                stats_train["prompt_end_kept"].append(0)
                stats_train["prompt_middle_removed"].append(0)
        else:
            prompt_was_truncated = True
            prompt_was_truncated_middle = True
            keep_start = int(available_for_prompt * start_ratio)
            keep_end = available_for_prompt - keep_start
            final_prompt_ids = prompt_ids[:keep_start] + prompt_ids[-keep_end:]

            if collect_stats:
                middle_removed = original_prompt_len - (keep_start + keep_end)
                stats_train["prompt_start_kept"].append(keep_start)
                stats_train["prompt_end_kept"].append(keep_end)
                stats_train["prompt_middle_removed"].append(middle_removed)

        if collect_stats:
            if prompt_was_truncated:
                stats_train["prompt_truncated_count"] += 1
            if prompt_was_truncated_middle:
                stats_train["prompt_truncated_middle_count"] += 1
            if response_was_truncated:
                stats_train["response_truncated_count"] += 1

            stats_train["prompt_tokens_used"].append(len(final_prompt_ids))
            stats_train["response_tokens_used"].append(len(final_response_ids))

        if len(final_response_ids) < MIN_RESPONSE_TOKENS:
            continue

        input_ids = final_prompt_ids + final_response_ids
        labels = [-100] * len(final_prompt_ids) + final_response_ids
        attention_mask = [1] * len(input_ids)

        model_inputs["input_ids"].append(input_ids)
        model_inputs["attention_mask"].append(attention_mask)
        model_inputs["labels"].append(labels)
        model_inputs["source_texts"].append(source_text)

    return model_inputs

# ============================================================================
# TOKENIZAR (asume que hf_ds ya está cargado desde tu código)
# ============================================================================

print("\n🔄 Tokenizando datasets...")

tokenized_train = hf_ds["train"].map(
    lambda x: preprocess_function(x, collect_stats=True),
    batched=True,
    remove_columns=hf_ds["train"].column_names,
    desc="Tokenizing train"
)

tokenized_test = hf_ds["test"].map(
    lambda x: preprocess_function(x, collect_stats=False),
    batched=True,
    remove_columns=hf_ds["test"].column_names,
    desc="Tokenizing test"
)

# Filtrar ejemplos sin labels válidos
print("\n🔧 Filtrando ejemplos sin labels válidos...")

def has_valid_labels(example):
    return sum(1 for l in example['labels'] if l != -100) > 0

tokenized_train_original = tokenized_train
tokenized_train = tokenized_train.filter(has_valid_labels)

# ============================================================================
# 🔍 DIAGNÓSTICO COMPLETO DEL DATASET
# ============================================================================

print("\n" + "="*75)
print("🔍 VERIFICACIÓN DE EJEMPLOS INDIVIDUALES")
print("="*75)

# Verificar los primeros 10 ejemplos
for i in range(min(10, len(tokenized_train))):
    example = tokenized_train[i]

    total_tokens = len(example['input_ids'])
    valid_tokens = sum(1 for l in example['labels'] if l != -100)
    prompt_tokens = sum(1 for l in example['labels'] if l == -100)

    print(f"\nEjemplo {i}:")
    print(f"  Total: {total_tokens} tokens")
    print(f"  Prompt (-100): {prompt_tokens} tokens ({prompt_tokens/total_tokens*100:.1f}%)")
    print(f"  Response (válidos): {valid_tokens} tokens ({valid_tokens/total_tokens*100:.1f}%)")

    if valid_tokens == 0:
        print(f"  ⚠️ PROBLEMA: Este ejemplo tiene 0 tokens válidos!")
        print(f"  Labels únicos: {set(example['labels'])}")

        # Decodificar para ver qué tiene
        text = tokenizer.decode(example['input_ids'][:200], skip_special_tokens=True)
        print(f"  Texto (primeros 200 chars): {text[:200]}...")

    # Verificar si tiene source_texts
    if 'source_texts' in example:
        print(f"  ✅ source_texts presente: {len(example['source_texts'])} chars")
    else:
        print(f"  ❌ source_texts AUSENTE")

print("="*75)

# Contar cuántos ejemplos tienen 0 tokens válidos
print("\n📊 ANÁLISIS COMPLETO DEL DATASET:")
print("-" * 75)

zero_valid = 0
valid_count = []

for ex in tokenized_train:
    valid = sum(1 for l in ex['labels'] if l != -100)
    valid_count.append(valid)
    if valid == 0:
        zero_valid += 1

print(f"Total ejemplos: {len(tokenized_train)}")
print(f"Ejemplos con 0 tokens válidos: {zero_valid} ({zero_valid/len(tokenized_train)*100:.1f}%)")
print(f"Ejemplos OK: {len(tokenized_train) - zero_valid} ({(len(tokenized_train) - zero_valid)/len(tokenized_train)*100:.1f}%)")

if len(valid_count) > 0:
    import numpy as np
    print(f"\nEstadísticas de tokens válidos:")
    print(f"  Mínimo: {min(valid_count)}")
    print(f"  Máximo: {max(valid_count)}")
    print(f"  Promedio: {np.mean(valid_count):.1f}")
    print(f"  Mediana: {np.median(valid_count):.1f}")

if zero_valid > 0:
    print(f"\n⚠️ PROBLEMA DETECTADO: Hay {zero_valid} ejemplos sin tokens válidos!")
    print(f"   El filtro has_valid_labels() debería haberlos eliminado.")
    print(f"\n💡 Solución: Re-aplicar el filtro:")
    print(f"   tokenized_train = tokenized_train.filter(lambda x: sum(1 for l in x['labels'] if l != -100) > 0)")
else:
    print(f"\n✅ TODOS los ejemplos tienen tokens válidos!")

# Verificar columnas
print(f"\n📋 Columnas del dataset:")
print(f"  {tokenized_train.column_names}")

tokenized_test_original = tokenized_test
tokenized_test = tokenized_test.filter(has_valid_labels)

print(f"✅ Filtrado completado:")
print(f"   Train: {len(tokenized_train_original)} → {len(tokenized_train)} ejemplos")
print(f"   Test: {len(tokenized_test_original)} → {len(tokenized_test)} ejemplos")

# ESTADÍSTICAS
total_examples = len(stats_train['prompt_tokens_original'])

print("\n" + "="*75)
print("📊 ESTADÍSTICAS DE TRAIN")
print("="*75)
print(f"Total ejemplos: {total_examples}")
print(f"Ejemplos completos: {stats_train['both_fit_count']} ({stats_train['both_fit_count']/total_examples*100:.1f}%)")

avg_prompt_orig = sum(stats_train['prompt_tokens_original'])/len(stats_train['prompt_tokens_original'])
avg_prompt_used = sum(stats_train['prompt_tokens_used'])/len(stats_train['prompt_tokens_used'])
avg_response_orig = sum(stats_train['response_tokens_original'])/len(stats_train['response_tokens_original'])
avg_response_used = sum(stats_train['response_tokens_used'])/len(stats_train['response_tokens_used'])

print(f"\n📝 PROMPT:")
print(f"  - Tokens promedio: {avg_prompt_used:.0f}")
print(f"\n💬 RESPONSE:")
print(f"  - Tokens promedio: {avg_response_used:.0f}")

avg_total = avg_prompt_used + avg_response_used
print(f"\n📏 USO DE SECUENCIA:")
print(f"  - Total usado: {avg_total:.0f} tokens ({avg_total/2048*100:.1f}%)") ###2048
print(f"  - Espacio libre: {2048 - avg_total:.0f} tokens") ###2048
print("="*75 + "\n")

# ============================================================================
# DATA COLLATOR
# ============================================================================
# 🔍 DIAGNÓSTICO: Ver qué hay en el dataset después de tokenizar
print("\n" + "="*75)
print("🔍 DIAGNÓSTICO DEL DATASET")
print("="*75)

# Ver columnas disponibles
print(f"\n📊 Columnas en tokenized_train:")
print(f"   {tokenized_train.column_names}")

# Ver un ejemplo completo
if len(tokenized_train) > 0:
    sample = tokenized_train[0]
    print(f"\n📋 Claves del ejemplo 0:")
    for key in sample.keys():
        if key == "source_texts":
            print(f"   ✅ {key}: {sample[key][:100]}..." if isinstance(sample[key], str) else f"   ✅ {key}: presente")
        else:
            print(f"   - {key}: type={type(sample[key])}, len={len(sample[key]) if hasattr(sample[key], '__len__') else 'N/A'}")

    # Verificar si source_texts existe
    if "source_texts" not in sample:
        print(f"\n❌ PROBLEMA: 'source_texts' NO está en el dataset!")
        print(f"   El dataset solo tiene: {list(sample.keys())}")
    else:
        print(f"\n✅ OK: 'source_texts' está presente")

print("="*75 + "\n")

# ============================================================================
# DATA COLLATOR CON SOURCE_TEXTS (para cualquier batch_size)
# ============================================================================

from dataclasses import dataclass
from typing import Dict, List
import torch

@dataclass
class DataCollatorWithSourceTexts:
    def __init__(self, tokenizer, pad_to_multiple_of=None):
        self.tokenizer = tokenizer
        self.pad_to_multiple_of = pad_to_multiple_of

    def __call__(self, features):
        #print(f"[COLLATOR] ✅ Llamado con {len(features)} ejemplos.")

        if not features:
            return {
                "input_ids": torch.empty(0, dtype=torch.long),
                "attention_mask": torch.empty(0, dtype=torch.long),
                "labels": torch.empty(0, dtype=torch.long),
                "source_texts": []
            }

        # 1️⃣ Extraer textos y limpiar features
        source_texts = [f.get("source_texts", "") for f in features]
        cleaned_features = [{k: v for k, v in f.items() if k != "source_texts"} for f in features]

        # 2️⃣ Padding manual
        max_length = max(len(f["input_ids"]) for f in cleaned_features)
        if self.pad_to_multiple_of:
            max_length = ((max_length + self.pad_to_multiple_of - 1)
                          // self.pad_to_multiple_of * self.pad_to_multiple_of)

        batch = {"input_ids": [], "attention_mask": [], "labels": []}

        for f in cleaned_features:
            pad_len = max_length - len(f["input_ids"])
            batch["input_ids"].append(f["input_ids"] + [self.tokenizer.pad_token_id] * pad_len)
            batch["attention_mask"].append(f["attention_mask"] + [0] * pad_len)
            batch["labels"].append(f["labels"] + [-100] * pad_len)

        # 3️⃣ Convertir a tensores
        batch["input_ids"] = torch.tensor(batch["input_ids"], dtype=torch.long)
        batch["attention_mask"] = torch.tensor(batch["attention_mask"], dtype=torch.long)
        batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)

        # ⚡ 4️⃣ Añadir source_texts como metadato, fuera del to(device)
        batch["metadata"] = {"source_texts": source_texts}

        return batch


# Crear data collator
data_collator = DataCollatorWithSourceTexts(
    tokenizer=tokenizer,
    pad_to_multiple_of=8
)

print("✅ Data collator configurado (soporta cualquier batch_size)")


# ============================================================================
# CONFIGURACIÓN
# ============================================================================

from transformers import Trainer, TrainingArguments


# ============================================================================
# PASO 2: Trainer personalizado
# ============================================================================

class CosineAlignedTrainer(Trainer):
    def __init__(self,
                 *args,
                 cosine_weight=0.2,
                 kl_weight=0.02,   # <<--- añadir aquí
                 hidden_size=2048,
                 embedding_model=None,
                 embedding_tokenizer=None,
                 proj=None,
                 ref_model=None,
                 teacher_cpu=True,
                 readability_weight=0,
                 complexity_weight=0,
                 **kwargs):

        super().__init__(*args, **kwargs)

        self.loss_fn = CosineAlignedLoss(
            embedding_model=embedding_model,
            embedding_tokenizer=embedding_tokenizer,
            cosine_weight=cosine_weight,
            kl_weight=kl_weight,    # <<--- usar aquí
            readability_weight=readability_weight,
            complexity_weight=complexity_weight,
            proj=proj,
            ref_model=ref_model

        )

        self.cosine_weight = cosine_weight
        self.readability_weight = readability_weight  #
        self.complexity_weight = complexity_weight    #
        self.teacher_cpu = teacher_cpu
        self.ref_model = ref_model

        # Listas de tracking (añadir nuevas)
        self.ce_losses = []
        self.cosine_losses = []
        self.kl_losses = []            #
        self.readability_losses = []   #
        self.complexity_losses = []    #
        self.total_losses = []

        self.gpu_peak_memory = 0
        self._debug_count = 0

        # 🧊 Congelar modelo de embeddings (MiniLM)
        if embedding_model is not None:
            embedding_model.eval()
            for p in embedding_model.parameters():
                p.requires_grad = False

        # 🧊 Congelar el modelo teacher (referencia)
        if ref_model is not None:
            ref_model.eval()
            for p in ref_model.parameters():
                p.requires_grad = False
            self._teacher_device = next(ref_model.parameters()).device

    # ================================================================
    # 🔁 Forward del teacher según configuración (CPU o GPU temporal)
    # ================================================================
    def _teacher_forward(self, inputs, device):
        try:
            if self.teacher_cpu:
                inputs_cpu = {k: v.to("cpu") for k, v in inputs.items()}
                ref_out = self.ref_model(**inputs_cpu, return_dict=True)
                return ref_out.logits.to(device)
            else:
                inputs_gpu = {k: v.to(self._teacher_device) for k, v in inputs.items()}
                ref_out = self.ref_model(**inputs_gpu, return_dict=True)
                return ref_out.logits.to(device)
        except Exception as e:
            print(f"[❌ KL ERROR] Teacher forward failed: {e}", file=sys.stderr)
            return None

    # ================================================================
    # 🧮 Cálculo de la pérdida total (CE + Cosine + KL)
    # ================================================================
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):

      # ---------------------------------------------------------
      # 1) Extraer textos originales del batch
      # ---------------------------------------------------------
      if "metadata" in inputs and "source_texts" in inputs["metadata"]:
          source_texts = inputs["metadata"]["source_texts"]
      elif "source_texts" in inputs:
          source_texts = inputs["source_texts"]
      else:
          source_texts = []
          print("⚠️ No hay 'source_texts' en este batch", file=sys.stderr)

      labels = inputs.get("labels")

      # ---------------------------------------------------------
      # 2) Limpiar inputs (solo tensores)
      # ---------------------------------------------------------
      clean_inputs = {
          k: v for k, v in inputs.items()
          if isinstance(v, torch.Tensor)
      }

      # ---------------------------------------------------------
      # 3) Forward del STUDENT (con hidden si se usa cosine/read/complexity)
      # ---------------------------------------------------------
      use_hidden = (
          self.cosine_weight > 0 or
          self.readability_weight > 0 or
          self.complexity_weight > 0
      )

      outputs = model(
          **clean_inputs,
          output_hidden_states=use_hidden,
          return_dict=True
      )

      # ---------------------------------------------------------
      # 4) Calcular pérdidas: CE + Cos + KL + Read + Complex
      # ---------------------------------------------------------
      (
          total_loss,
          ce_loss,
          cosine_loss,
          kl_loss,
          readability_loss,
          complexity_loss
      ) = self.loss_fn(
          outputs=outputs,
          labels=labels,
          texts=source_texts,
          inputs=clean_inputs,
          tokenizer=self.tokenizer
      )

      # ---------------------------------------------------------
      # 5) Logging interno del trainer
      # ---------------------------------------------------------
      self.ce_losses.append(float(ce_loss))
      self.cosine_losses.append(float(cosine_loss))
      self.kl_losses.append(float(kl_loss))
      self.readability_losses.append(float(readability_loss))
      self.complexity_losses.append(float(complexity_loss))
      self.total_losses.append(float(total_loss))

      # DEBUG bootstrap
      if self._debug_count < 3:
          print(
              f"CE={float(ce_loss):.4f} | "
              f"Cos={float(cosine_loss):.4f} | "
              f"KL={float(kl_loss):.4f} | "
              f"READ={float(readability_loss):.4f} | "
              f"COMP={float(complexity_loss):.4f} | "
              f"Total={float(total_loss):.4f}",
              file=sys.stderr
          )
          self._debug_count += 1

      # Logging periódico
      if self.state.global_step % 10 == 0:
          print(
              f"[Step {self.state.global_step:4d}] "
              f"CE={float(ce_loss):.4f} | "
              f"Cos={float(cosine_loss):.4f} | "
              f"KL={float(kl_loss):.4f} | "
              f"READ={float(readability_loss):.4f} | "
              f"COMP={float(complexity_loss):.4f} | "
              f"Total={float(total_loss):.4f}",
              file=sys.stderr
          )

      # -----------------------------------------
      #   🔥 Guardar TU loss en el log_history
      # -----------------------------------------
      self.state.log_history.append(
          {
              "step": int(self.state.global_step),
              "custom_total_loss": float(total_loss.detach().cpu()),
              "ce_loss": float(ce_loss.detach().cpu()),
              "kl_loss": float(kl_loss.detach().cpu()),
              "cosine_loss": float(cosine_loss.detach().cpu()),
              "readability_loss": float(readability_loss.detach().cpu()),
              "complexity_loss": float(complexity_loss.detach().cpu()),
          }
      )

      return (total_loss, outputs) if return_outputs else total_loss

# ============================================================================
# CREAR TRAINER
# ============================================================================

# Evitar que Hugging Face convierta automáticamente los campos string problema source text
tokenized_train.set_format(type="python")
tokenized_test.set_format(type="python")

# ============================================================================
# PASO 3: Setup completo
# ============================================================================

# 2. LoRA
print("\n🔧 Configurando LoRA...")
peft_config=LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
print(f"✅ LoRA configurado")

# 3. Cargar embedding model
print("\n📥 Cargando embedding model (MiniLM)...")
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)
embedding_model = AutoModel.from_pretrained(embedding_model_name).to("cpu")
print("✅ Embedding model cargado en CPU")

# 4. Crear proyector EXTERNO
print("\n🔧 Creando proyector...")
cosine_proj = nn.Sequential(
    nn.Linear(2048, 512), # hidden_size para Llama 3.2-1B / 3B 3072
    nn.Dropout(0.1),
    nn.LayerNorm(512),
    nn.GELU(),
    nn.Linear(512, 384)
).cuda()

# ✅ CRÍTICO: Registrar proyector en el modelo
model.cosine_proj = cosine_proj
print("✅ Proyector registrado en model.cosine_proj")

# Verificar que tiene gradientes
for param in model.cosine_proj.parameters():
    param.requires_grad = True
print(f"✅ Proyector requiere gradientes: {next(model.cosine_proj.parameters()).requires_grad}")

# =============================================
# 🧠 Configuración del modelo de referencia (teacher) para KL
# =============================================

TEACHER_INIT_CPU = False    # True = inicia en CPU
TEACHER_CPU = False         # True = permanece en CPU, False = sube temporalmente a GPU

print("📥 Cargando teacher FP16 en GPU completa...")

ref_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,   # tu Llama 3.2 3B instruct
    torch_dtype=torch.float16,
    device_map="cuda",   # TODO EL MODELO EN GPU
)

ref_model.eval()
for p in ref_model.parameters():
    p.requires_grad = False

print("✅ Teacher FULL GPU listo")

print(f"Student device: {next(model.parameters()).device}")
print(f"Teacher device: {next(ref_model.parameters()).device}")

if hasattr(model, "cosine_proj"):
    print(f"Proyector device: {next(model.cosine_proj.parameters()).device}")
else:
    print("ℹ️ No hay proyector (proj=None)")


# 5. Training arguments====================================
print("\n🔧 Configurando training...")

training_args = TrainingArguments(
    output_dir="llama3_1B_cosine_fixed",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=48, ###
    num_train_epochs=num_train_epochs,
    learning_rate=5e-6 , ### probar 4e-5
    weight_decay=0.01,
    warmup_steps=0,  # ← CAMBIAR: 0 warmup steps
    # warmup_ratio=0.05,  # ← ELIMINAR esta línea
    optim="paged_adamw_8bit",
    max_grad_norm=0.3,
    save_strategy="epoch",
    save_total_limit=4, ### guarda cada epoch pero solo guarda los x ultimos, va borrando
    #eval_strategy="epoch",
    eval_strategy="no",
    #eval_strategy="steps",
    #eval_steps=20,
    logging_steps=5,
    logging_first_step=True,
    disable_tqdm=False,
    report_to="none",
    fp16=True, ###
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": True},
    dataloader_pin_memory=False,
    remove_unused_columns=False,  # ← ✅ CRÍTICO para source_texts
    #fp16=False,     # 🚫 Desactiva AMP desde el inicio
    #bf16=False,     # 🚫 idem
)

# 6. Crear trainer
print("\n🔧 Creando trainer...")


trainer = CosineAlignedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    #eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,  # ← Usar el nuevo
    cosine_weight=COSINE_WEIGHT,
    hidden_size=2048,  # Para 1B <----
    embedding_model=embedding_model,
    embedding_tokenizer=embedding_tokenizer,
    #proj=cosine_proj,
    teacher_cpu=TEACHER_CPU,   # 👈 aquí
    proj=None,
    ref_model=ref_model,
    kl_weight=KL_WEIGHT,
    readability_weight=READABILITY_WEIGHT,
    complexity_weight=COMPLEXITY_WEIGHT,
)

import warnings
from transformers.utils import logging

warnings.filterwarnings("ignore", message=".*Trainer.tokenizer is now deprecated.*")
logging.set_verbosity_error()


for name, param in model.named_parameters():
    if param.requires_grad:
        if param.grad is None:
            print(f"[⚠️] {name} sin gradiente (no se está actualizando)")
        else:
            print(f"[✅] {name} gradiente OK")
    else:
        print(f"[🧊] {name} congelado")
    break  # solo imprimimos el primero




#trainer.loss_fn.ref_model = ref_model
trainer.loss_fn.kl_weight = KL_WEIGHT  # empieza suave

# ✅ AHORA (funciona):
print("✅ Trainer configurado")

# Callback para verificar después del primer step
from transformers import TrainerCallback

class VerifyOptimizerCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step == 10:  # ← Cambiar de 1 a 10
            print("\n" + "="*80)
            print("🔍 VERIFICACIÓN DEL OPTIMIZER (step 10)")
            print("="*80)
            optimizer = kwargs['optimizer']
            print(f"📊 Total params: {sum(len(g['params']) for g in optimizer.param_groups)}")
            for i, group in enumerate(optimizer.param_groups):
                print(f"   Grupo {i}: LR={group['lr']:.2e}, Params={len(group['params'])}")
            print("="*80 + "\n")
        return control

trainer.add_callback(VerifyOptimizerCallback())



# ============================================================
# 🧠 Diagnóstico previo: ver qué genera el modelo antes de entrenar
# ============================================================
prompt = "### Original text: The patient reported chest pain. ### Output:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# ⚙️ Generar texto sin entrenar (modelo base o inicializado)
with torch.no_grad():
    out = model.generate(**inputs, max_new_tokens=50, temperature=0.8, top_p=0.9)
print("\n🧩 Generación inicial (modelo sin entrenar):")
print(tokenizer.decode(out[0], skip_special_tokens=False))
print("=" * 80)

print("🧠 Trainer class:", type(trainer))
print("🔍 compute_loss:", trainer.compute_loss.__qualname__)

# Entrenar
print("\n" + "="*80)
print("🚀 INICIANDO ENTRENAMIENTO")
print("="*80)


from transformers.utils import logging

warnings.filterwarnings("ignore", message=".*Trainer.tokenizer is now deprecated.*")
logging.set_verbosity_error()

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

trainer.train()

# 8. Diagnóstico final
print("\n" + "="*80)
print("🔍 DIAGNÓSTICO FINAL")
print("="*80)

# Guardar modelo
print("\n💾 Guardando modelo...")
model.save_pretrained("llama3_3B_cosine_final") ### cambiar nombre segun modelo
tokenizer.save_pretrained("llama3_3B_cosine_final")
print("✅ Modelo guardado")

print("\n" + "="*80)
print("🎉 ¡COMPLETADO!")
print("="*80)


🚀 SETUP - OPCIÓN C FIXED

📥 Cargando meta-llama/Llama-3.2-3B-Instruct...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

CE=3.9202 | Cos=0.0000 | KL=144.3848 | READ=0.0000 | COMP=0.0000 | Total=18.3587
[Step    0] CE=3.9202 | Cos=0.0000 | KL=144.3848 | READ=0.0000 | COMP=0.0000 | Total=18.3587
[Step    0] CE=3.7453 | Cos=0.0000 | KL=149.8198 | READ=0.0000 | COMP=0.0000 | Total=18.7272
[Step    0] CE=3.8434 | Cos=0.0000 | KL=149.1018 | READ=0.0000 | COMP=0.0000 | Total=18.7536
[Step    0] CE=4.0472 | Cos=0.0000 | KL=189.2894 | READ=0.0000 | COMP=0.0000 | Total=22.9762
[Step    0] CE=3.6027 | Cos=0.0000 | KL=164.2322 | READ=0.0000 | COMP=0.0000 | Total=20.0259
[Step    0] CE=3.3620 | Cos=0.0000 | KL=148.5923 | READ=0.0000 | COMP=0.0000 | Total=18.2212
[Step    0] CE=4.2082 | Cos=0.0000 | KL=155.3497 | READ=0.0000 | COMP=0.0000 | Total=19.7432
[Step    0] CE=3.6494 | Cos=0.0000 | KL=177.0722 | READ=0.0000 | COMP=0.0000 | Total=21.3567
[Step    0] CE=3.6725 | Cos=0.0000 | KL=177.8245 | READ=0.0000 | COMP=0.0000 | Total=21.4550
[Step    0] CE=3.8794 | Cos=0.0000 | KL=124.2165 | READ=0.0000 | COMP=0.0000 | Tot

Step,Training Loss
1,971.928500
5,973.879000
10,983.680200
15,961.998200
20,952.552800
25,941.097400
30,927.193600
35,900.986400
40,804.841300
45,883.553000



🔍 VERIFICACIÓN DEL OPTIMIZER (step 10)
📊 Total params: 230
   Grupo 0: LR=4.80e-06, Params=226
   Grupo 1: LR=4.80e-06, Params=4



[Step   10] CE=3.7321 | Cos=0.0000 | KL=198.2443 | READ=0.0000 | COMP=0.0000 | Total=23.5565
[Step   10] CE=3.9141 | Cos=0.0000 | KL=213.6948 | READ=0.0000 | COMP=0.0000 | Total=25.2836
[Step   10] CE=3.9370 | Cos=0.0000 | KL=149.8016 | READ=0.0000 | COMP=0.0000 | Total=18.9171
[Step   10] CE=3.9728 | Cos=0.0000 | KL=108.3764 | READ=0.0000 | COMP=0.0000 | Total=14.8105
[Step   10] CE=3.7348 | Cos=0.0000 | KL=163.7262 | READ=0.0000 | COMP=0.0000 | Total=20.1074
[Step   10] CE=3.8679 | Cos=0.0000 | KL=143.9701 | READ=0.0000 | COMP=0.0000 | Total=18.2649
[Step   10] CE=3.8354 | Cos=0.0000 | KL=162.0004 | READ=0.0000 | COMP=0.0000 | Total=20.0355
[Step   10] CE=3.6864 | Cos=0.0000 | KL=160.2084 | READ=0.0000 | COMP=0.0000 | Total=19.7073
[Step   10] CE=3.8681 | Cos=0.0000 | KL=157.1240 | READ=0.0000 | COMP=0.0000 | Total=19.5805
[Step   10] CE=4.0187 | Cos=0.0000 | KL=149.8192 | READ=0.0000 | COMP=0.0000 | Total=19.0006
[Step   10] CE=3.7854 | Cos=0.0000 | KL=177.8329 | READ=0.0000 | COMP=


🔍 DIAGNÓSTICO FINAL
✅ Cosine Loss evolution:
   Inicial: 0.0000
   Final: 0.0000
   Bajó: 0.0000

📊 GPU Peak: 0.00 GB

💾 Guardando modelo...
✅ Modelo guardado

🎉 ¡COMPLETADO!


In [ ]:
print(f"Dataset original size:")
print(f"   Train: {len(hf_ds['train'])}")
print(f"   Test: {len(hf_ds['test'])}")

Dataset original size:
   Train: 1800
   Test: 200


In [ ]:
prompt = "Input abstract: The feline consumed the rodent. Output PLS:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = trainer.model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Input abstract: The feline consumed the rodent. Output PLS: TheimportQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestionQuestion


Si hay que continuar más Epoch se EJECUTA esto

In [ ]:
'''trainer.args.num_train_epochs = 6 # hasta 6
trainer.train(resume_from_checkpoint=True)
'''

Step,Training Loss
155,790.636400
160,768.926600
165,767.618000
170,777.641800
175,763.070300
180,761.291600
185,755.434300
190,676.083000
195,757.799500
200,751.764400


[Step  160] CE=3.4532 | Cos=0.0000 | KL=148.7166 | READ=0.0000 | COMP=0.0000 | Total=18.3249
[Step  160] CE=3.6150 | Cos=0.0000 | KL=123.6665 | READ=0.0000 | COMP=0.0000 | Total=15.9816
[Step  160] CE=3.7670 | Cos=0.0000 | KL=113.9188 | READ=0.0000 | COMP=0.0000 | Total=15.1589
[Step  160] CE=4.0123 | Cos=0.0000 | KL=111.2677 | READ=0.0000 | COMP=0.0000 | Total=15.1391
[Step  160] CE=3.8403 | Cos=0.0000 | KL=155.3726 | READ=0.0000 | COMP=0.0000 | Total=19.3775
[Step  160] CE=3.5364 | Cos=0.0000 | KL=133.5824 | READ=0.0000 | COMP=0.0000 | Total=16.8946
[Step  160] CE=3.3456 | Cos=0.0000 | KL=125.5830 | READ=0.0000 | COMP=0.0000 | Total=15.9039
[Step  160] CE=3.6220 | Cos=0.0000 | KL=135.2389 | READ=0.0000 | COMP=0.0000 | Total=17.1459
[Step  160] CE=3.9384 | Cos=0.0000 | KL=103.9791 | READ=0.0000 | COMP=0.0000 | Total=14.3363
[Step  160] CE=3.8936 | Cos=0.0000 | KL=128.6964 | READ=0.0000 | COMP=0.0000 | Total=16.7633
[Step  160] CE=3.8166 | Cos=0.0000 | KL=113.6065 | READ=0.0000 | COMP=

TrainOutput(global_step=228, training_loss=248.98100575229577, metrics={'train_runtime': 5826.1215, 'train_samples_per_second': 1.854, 'train_steps_per_second': 0.039, 'total_flos': 2.7746317042886246e+17, 'train_loss': 248.98100575229577, 'epoch': 6.0})

## Fin de training

In [ ]:
logs = pd.DataFrame(trainer.state.log_history)
print(logs.head())

NameError: name 'trainer' is not defined

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# -------------------------
# Training Loss
# -------------------------
train_mask = logs["loss"].notna() & logs["step"].notna()
steps_train = logs.loc[train_mask, "step"].values
loss_train = logs.loc[train_mask, "loss"].values

# -------------------------
# Eval Loss (si existe)
# -------------------------
if "eval_loss" in logs.columns:
    eval_mask = logs["eval_loss"].notna() & logs["step"].notna()
    steps_eval = logs.loc[eval_mask, "step"].values
    loss_eval = logs.loc[eval_mask, "eval_loss"].values
else:
    steps_eval, loss_eval = [], []
    print("⚠️ No hay columna eval_loss en logs")


plt.figure(figsize=(10, 6))

# === TRAINING ===
plt.plot(
    steps_train, loss_train,
    label="Training loss",
    linewidth=2.2,
    color='#2E86AB',
    alpha=0.9,
    marker='o',
    markersize=4,
    markerfacecolor='white',
    markeredgewidth=1.2,
    markeredgecolor='#2E86AB'
)

# === EVAL ===
if len(steps_eval) > 0:
    plt.plot(
        steps_eval, loss_eval,
        label="Eval loss",
        linewidth=2.2,
        color='#E74C3C',
        alpha=0.9,
        marker='s',
        markersize=4,
        markerfacecolor='white',
        markeredgewidth=1.2,
        markeredgecolor='#E74C3C'
    )

plt.xlabel("Iteración (step)", fontsize=13, fontweight='bold')
plt.ylabel("Loss", fontsize=13, fontweight='bold')
plt.title("Curva de pérdida durante el entrenamiento", fontsize=15, fontweight='bold', pad=20)
plt.legend(fontsize=12, frameon=True, shadow=True)
plt.grid(True, alpha=0.3, linestyle='--', linewidth=0.7)
plt.tight_layout()
plt.show()

print(f"✅ Training points: {len(steps_train)}")
if len(steps_eval) > 0:
    print(f"🔍 Eval points: {len(steps_eval)}")


NameError: name 'logs' is not defined

## Guardar en Drive los resultados del modelo

In [ ]:
# 1. Guardar modelo y tokenizer
#trainer.save_model("llama3_lora_simplification_final")
# Guardar local
trainer.model.save_pretrained("llama3_2_3b_pls_lora/adapter")

# Definir versión
VERSION = "v6_L4_6_epochs_CE_KL_PROMPT1_2000_instruct_Low_v2"  # ← CAMBIAR ESTO CADA VEZ

# Copiar a Drive
!mkdir -p /content/drive/MyDrive/llama3_models
!cp -r llama3_2_3b_pls_lora/adapter /content/drive/MyDrive/llama3_models/adapter_{VERSION}

print(f"✅ Guardado: adapter_{VERSION}")
#tokenizer.save_pretrained("llama3_lora_simplification_final")
print("✅ Modelo guardado")

## Guardar en Drive los checkpoints - de epoch intermedios

In [ ]:
!mkdir -p "/content/drive/MyDrive/llama_models/Instruct_Low_KL_frozen" ### Cambiar Nombre
!cp -r "/content/cochrane_data/llama3_1B_cosine_fixed" "/content/drive/MyDrive/llama_models/Instruct_Low_KL_frozen" ### Cambiar Nombre

# solo si hay que Recuperar modelo

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

In [ ]:
BASE_MODEL = "meta-llama/Llama-3.2-3B-Instruct"  # o el modelo base que usaste originalmente
ADAPTER_PATH = "/content/drive/MyDrive/llama3_models/adapter_v6_T4_2_epochs_CE_PROMPT1_2000_instruct"

# 1. Cargar modelo base
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype="auto",
    device_map="auto"
)

# 2. Aplicar el LoRA adapter
from peft import PeftModel
model = PeftModel.from_pretrained(model, ADAPTER_PATH)

# 3. Cargar el tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

print("✅ Modelo y tokenizer cargados con el adapter LoRA aplicado.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

✅ Modelo y tokenizer cargados con el adapter LoRA aplicado.


# 4.Generación de Resumenes


## Preparar Dataset para pasar a DataFrame df_test para la Generación de Texto y Evaluacion de métricas

Elimina promp y crea source

In [ ]:
print(hf_ds["test"].column_names)
df_test = hf_ds["test"].to_pandas()
print(df_test.columns)

import re
import pandas as pd

def extract_source_from_prompt(prompt):
    """
    Extrae el texto entre 'original text' y 'simplified text' del prompt.
    Robusto ante espacios, mayúsculas y saltos de línea.
    """
    match = re.search(
        r"Input abstract:\s*[:\-–]?\s*(.*?)\s*Output PLS",
        prompt,
        re.IGNORECASE | re.DOTALL
    )
    if match:
        return match.group(1).strip()
    return ""

# Crear columna 'source' desde 'prompt'
df_test["source"] = df_test["prompt"].apply(extract_source_from_prompt)

# Mantener solo columnas necesarias
df_test = df_test[["source", "response"]]

# Mostrar ejemplo de verificación
print(df_test.head(2))

['prompt', 'response']
Index(['prompt', 'response'], dtype='object')
                                              source  \
0  Background\nPeriodic fever, aphthous stomatiti...   
1  Background\nMemory problems are a common cogni...   

                                            response  
0  Tonsillectomy for PFAPA syndrome (a rare cause...  
1  Cognitive rehabilitation for memory deficits a...  


## Generación de texto para el set TEST

Asigna el modelo del trainer y del modelo cargado según lo que aplique

In [ ]:
if "trainer" in locals() and hasattr(trainer, "model"):
    fine_tuned_model = trainer.model.eval()
else:
    fine_tuned_model = model.eval()

In [ ]:
import gc
import torch
from tqdm import tqdm
from difflib import SequenceMatcher
import re

def generate_simplifications_v2(
    df,
    model,
    tokenizer,
    instr_prompt,
    batch_size=2,
    max_source_tokens=512,
    sim_threshold=0.75,
    verbose=True,
):
    """
    Genera textos simplificados con regeneración multinivel.
    Si el texto generado está vacío, intenta regenerar automáticamente.
    """

    import gc, torch, re
    from tqdm import tqdm
    from difflib import SequenceMatcher

    tokenizer.padding_side = "right"
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.clean_up_tokenization_spaces = True

    model.eval()
    model.config.use_cache = True
    model.config.output_hidden_states = False
    model.config.return_dict = True

    torch.cuda.empty_cache()
    gc.collect()

    if verbose:
        print("\n🚀 Generación iniciada (modo seguro, con detección de vacíos)...")

    # === FUNCIONES AUXILIARES ===
    def truncate_text(text, max_tokens=max_source_tokens):
        tokens = text.split()
        if len(tokens) <= max_tokens:
            return text
        head = tokens[:max_tokens // 2]
        tail = tokens[-max_tokens // 2:]
        return " ".join(head + ["[...]"] + tail)

    def safe_generate(prompts, gen_params):
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1536
        ).to(model.device)

        with torch.inference_mode():
            outputs = model.generate(**inputs, **gen_params)

        texts = tokenizer.batch_decode(
            outputs,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
            errors="replace"
        )
        return texts

    def extract_simplified_text(text):
        match = re.search(r"Output PLS\s*[:\-–]?\s*(.*)", text, re.IGNORECASE | re.DOTALL)
        if match:
            simplified = match.group(1).strip()
        else:
            simplified = text.strip()
        simplified = re.sub(r"<\|endoftext\|>", "", simplified)
        simplified = re.sub(r"\s+", " ", simplified).strip()
        return simplified

    # === PARÁMETROS DE GENERACIÓN ===
    GEN_PARAMS_L1 = {"max_new_tokens": 256, "temperature": 0.7, "top_p": 0.9, "do_sample": False,
                     "num_beams": 1, "repetition_penalty": 1.1,
                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.pad_token_id}

    GEN_PARAMS_L2 = {"max_new_tokens": 256, "temperature": 1.1, "top_p": 0.85, "do_sample": True,
                     "num_beams": 1, "repetition_penalty": 1.0,
                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.pad_token_id}

    GEN_PARAMS_L3 = {"max_new_tokens": 256, "temperature": 0.7, "top_p": 0.9, "do_sample": False,
                     "num_beams": 4, "repetition_penalty": 1.1,
                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.pad_token_id}

    # === PREPARACIÓN ===
    df = df.copy()
    df["source_for_generation"] = df["source"].apply(lambda x: truncate_text(x))
    if verbose:
        print("✅ Columna 'source_for_generation' creada correctamente")

    # === GENERACIÓN PRINCIPAL ===
    generated_texts, is_copy_list, was_regenerated_list = [], [], []
    regeneration_level_list, similarity_scores = [], []
    regeneration_count = 0

    for i in tqdm(range(0, len(df), batch_size), desc="Generando simplificaciones"):
        if i % 10 == 0:
            torch.cuda.empty_cache()
            gc.collect()

        batch_sources = df["source"].iloc[i:i+batch_size].tolist()
        batch_sources_trunc = df["source_for_generation"].iloc[i:i+batch_size].tolist()
        batch_prompts = [instr_prompt.format(source=s) for s in batch_sources_trunc]

        full_texts = safe_generate(batch_prompts, GEN_PARAMS_L1)

        for j, full_text in enumerate(full_texts):
            simplified = extract_simplified_text(full_text)
            source_idx = i + j
            if source_idx >= len(df):
                continue

            # 🔍 --- NUEVO BLOQUE: REGENERACIÓN SI EL TEXTO ESTÁ VACÍO ---
            if not simplified or len(simplified.strip()) == 0:
                if verbose:
                    print(f"⚠️ Texto vacío detectado en índice {source_idx}. Regenerando...")
                single_prompt = instr_prompt.format(source=batch_sources_trunc[j])
                regen_text = safe_generate([single_prompt], GEN_PARAMS_L2)[0]
                simplified = extract_simplified_text(regen_text)
                # Si sigue vacío, prueba con L3
                if not simplified or len(simplified.strip()) == 0:
                    beam_text = safe_generate([single_prompt], GEN_PARAMS_L3)[0]
                    simplified = extract_simplified_text(beam_text)

            # === Calcular similitud y regeneraciones ===
            sim = SequenceMatcher(None, batch_sources[j].lower(), simplified.lower()).ratio()
            is_copy = sim > sim_threshold
            regen_level = 0

            if is_copy:
                regeneration_count += 1
                single_prompt = instr_prompt.format(source=batch_sources_trunc[j])

                regen_text = safe_generate([single_prompt], GEN_PARAMS_L2)[0]
                simplified_regen = extract_simplified_text(regen_text)
                sim_regen = SequenceMatcher(None, batch_sources[j].lower(), simplified_regen.lower()).ratio()

                if sim_regen <= sim_threshold:
                    simplified, sim, is_copy, regen_level = simplified_regen, sim_regen, False, 2
                else:
                    beam_text = safe_generate([single_prompt], GEN_PARAMS_L3)[0]
                    simplified_beam = extract_simplified_text(beam_text)
                    sim_beam = SequenceMatcher(None, batch_sources[j].lower(), simplified_beam.lower()).ratio()

                    if sim_beam <= sim_threshold:
                        simplified, sim, is_copy, regen_level = simplified_beam, sim_beam, False, 3
                    else:
                        if sim_beam < sim_regen:
                            simplified, sim, regen_level = simplified_beam, sim_beam, 3
                        else:
                            simplified, sim, regen_level = simplified_regen, sim_regen, 2

            generated_texts.append(simplified)
            is_copy_list.append(is_copy)
            was_regenerated_list.append(regen_level > 0)
            regeneration_level_list.append(regen_level)
            similarity_scores.append(sim)

    # === GUARDAR RESULTADOS ===
    df["generated_text"] = generated_texts
    df["is_copy"] = is_copy_list
    df["was_regenerated"] = was_regenerated_list
    df["regeneration_level"] = regeneration_level_list
    df["similarity_score"] = similarity_scores

    if verbose:
        print(f"\n✅ Generación completada sin errores de memoria")
        print(f"📊 Total textos: {len(df)} | Regenerados: {sum(was_regenerated_list)} | Copias finales: {sum(is_copy_list)}")
        print(f"Promedio de similitud: {sum(similarity_scores)/len(similarity_scores):.3f}")

    return df


## Necesita source y response (filtro a 50 por rapidez)

In [ ]:
df_test50= df_test.sample(n=50, random_state=42)

print(df_test50.head())
print(len(df_test50))

                                                source  \
95   Background\nThis updated Cochrane Review of re...   
15   Background\nIn many countries emergency depart...   
30   Background\nPsychotherapy is regarded as the f...   
158  Background\nSpina bifida is a fetal neural tub...   
128  We included 12 trials in the original review (...   

                                              response  
95   Reminiscence therapy for dementia\nReview ques...  
15   Primary care professionals providing non‐urgen...  
30   Psychological therapies for borderline persona...  
158  Spina bifida repair and infant and maternal he...  
128  Transcutaneous Electrical Nerve Stimulation (T...  
50


In [ ]:
df_out = generate_simplifications_v2(
    df=df_test50,
    model=fine_tuned_model,
    tokenizer=tokenizer,
    instr_prompt=INSTR_PROMPT,
    batch_size=2,
)


🚀 Generación iniciada (modo seguro, con detección de vacíos)...
✅ Columna 'source_for_generation' creada correctamente


Generando simplificaciones:  32%|███▏      | 8/25 [04:09<08:47, 31.03s/it]

In [ ]:
ref = "PLS_CE_KL_Instruct_Llama__2000_6epoch_low"
filename = f"pls_results_{ref}.csv"
save_path = f"/content/drive/MyDrive/{filename}"

df_out.to_csv(save_path, index=False, encoding="utf-8")
print(f"📁 Archivo guardado en: {save_path}")

📁 Archivo guardado en: /content/drive/MyDrive/pls_results_PLS_CE_KL_Instruct_Llama__2000_2epoch_low.csv


In [ ]:
'''import pandas as pd

df_out = pd.read_csv("/content/drive/MyDrive/pls_results_PLS_CE_KL_Instruct_Llama__2000_10epoch_low.csv")
df_out.head()
'''

,source,response,source_for_generation,generated_text,is_copy,was_regenerated,regeneration_level,similarity_score
0,Background\nThis updated Cochrane Review of re...,Reminiscence therapy for dementia\nReview ques...,Background This updated Cochrane Review of rem...,**Plain Title** Revisiting Reminiscence Therap...,False,False,0,0.004935
1,Background\nIn many countries emergency depart...,Primary care professionals providing non‐urgen...,Background In many countries emergency departm...,**Plain Title** Reducing Overcrowding in Emerg...,False,False,0,0.072208
2,Background\nPsychotherapy is regarded as the f...,Psychological therapies for borderline persona...,Background Psychotherapy is regarded as the fi...,**Plain Title** Borderline Personality Disorde...,False,False,0,0.008960
3,Background\nSpina bifida is a fetal neural tub...,Spina bifida repair and infant and maternal he...,Background Spina bifida is a fetal neural tube...,**Plain Title** Spina Bifida Repair: Does Earl...,False,False,0,0.010991
4,We included 12 trials in the original review (...,Transcutaneous Electrical Nerve Stimulation (T...,We included 12 trials in the original review (...,**Plain Title** TENS Therapy for Acute Pain Re...,False,False,0,0.044089





# 5.Métricas


## Todas las métricas

In [ ]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00


In [ ]:
print(df_out.columns)

Index(['source', 'response', 'source_for_generation', 'generated_text',
       'is_copy', 'was_regenerated', 'regeneration_level', 'similarity_score',
       'generated_clean'],
      dtype='object')


In [ ]:
# ============================================================================
# 📊 PASO 1: CALCULAR MÉTRICAS (BERTScore + TextStat + AlignScore)
# ============================================================================

from bert_score import score as bertscore
import textstat
import pandas as pd
import torch
import transformers
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

EMB_MODEL = "sentence-transformers/all-mpnet-base-v2"  # cambia por modelo biomédico si quieres
DEVICE = "cuda" if (torch and torch.cuda.is_available()) else "cpu"
EMB_BATCH = 256   # reducir si OOM en T4 (prueba 128/64)

'''# Fix para AdamW
if not hasattr(transformers, "AdamW"):
    from torch.optim import AdamW
    transformers.AdamW = AdamW

from alignscore import AlignScore
'''

print("="*80)
print("📊 CALCULANDO MÉTRICAS")
print("="*80)

# --------------------------------------------------------
# 🔧 CONFIGURACIÓN INICIAL
# --------------------------------------------------------
col_ref = "response" #pls
col_gen = "generated_text"  # ← Cambié a generated_text (ajusta si es otro)
col_non_pls="source"

# Crear columna limpia si no existe
if col_gen not in df_out.columns and "generated_text" in df_test.columns:
    df_out["generated_clean"] = df_out["generated_text"].str.strip()
    col_gen = "generated_clean"

print(f"\n✓ Columna de referencia: {col_ref}")
print(f"✓ Columna generada: {col_gen}")
print(f"✓ Total textos: {len(df_out)}")

# --------------------------------------------------------
# 🧩 BERTScore
# --------------------------------------------------------
# --------------------------------------------------------
# 🧩 BERTScore 1 — modelo base (por defecto)
# --------------------------------------------------------
print("\n📊 Calculando BERTScore (modelo base)...")

refs = df_out[col_ref].astype(str).tolist()  # textos de referencia (PLS)
preds = df_out[col_gen].astype(str).tolist()  # textos generados
non_pls = df_out[col_non_pls].astype(str).tolist()  # abstract original

# Primer BERTScore: modelo por defecto ("roberta-large") o idioma "en"
P1, R1, F1_1 = bertscore(preds, refs, lang="en", verbose=False)

df_out["BERT_Precision"] = P1.numpy()
df_out["BERT_Recall"] = R1.numpy()
df_out["BERT_F1"] = F1_1.numpy()

print(f"✅ BERTScore (base) calculado")
print(f"   - Precision media: {P1.mean():.3f}")
print(f"   - Recall media: {R1.mean():.3f}")
print(f"   - F1 media: {F1_1.mean():.3f}")


# --------------------------------------------------------
# 🧩 BERTScore 2 — modelo alternativo o comparación distinta
# --------------------------------------------------------
print("\n📊 Calculando BERTScore (modelo alternativo: Longformer)...")

'''from evaluate import load
bertscore_alt = load("bertscore")

results_alt = bertscore_alt.compute(
    predictions=preds,
    references=refs,
    model_type="allenai/longformer-large-4096-finetuned-triviaqa",
    lang="en"
)

# Convertir los resultados en listas de precisión, recall y F1
P2 = results_alt["precision"]
R2 = results_alt["recall"]
F1_2 = results_alt["f1"]

# Guardar en df_out con nombres distintos
df_out["BERT2_Precision_Longformer"] = P2
df_out["BERT2_Recall_Longformer"] = R2
df_out["BERT2_F1_Longformer"] = F1_2

print(f"✅ BERTScore (Longformer) calculado")
print(f"   - Precision media: {sum(P2)/len(P2):.3f}")
print(f"   - Recall media: {sum(R2)/len(R2):.3f}")
print(f"   - F1 media: {sum(F1_2)/len(F1_2):.3f}")

'''

# --------------------------------------------------------
# 📚 TextStat (legibilidad y complejidad)
# --------------------------------------------------------
print("\n📚 Calculando métricas TextStat...")

def get_textstat_metrics(text):
    try:
        return {
            "flesch_reading_ease": textstat.flesch_reading_ease(text),
            "flesch_kincaid_grade": textstat.flesch_kincaid_grade(text),
            "smog_index": textstat.smog_index(text),
            "coleman_liau_index": textstat.coleman_liau_index(text),
            "automated_readability_index": textstat.automated_readability_index(text),
            "dale_chall_readability_score": textstat.dale_chall_readability_score(text),
            "difficult_words": textstat.difficult_words(text),
            "linsear_write_formula": textstat.linsear_write_formula(text),
            "gunning_fog": textstat.gunning_fog(text),
        }
    except:
        return {key: None for key in [
            "flesch_reading_ease", "flesch_kincaid_grade", "smog_index",
            "coleman_liau_index", "automated_readability_index",
            "dale_chall_readability_score", "difficult_words",
            "linsear_write_formula", "gunning_fog"
        ]}

textstat_metrics = df_out[col_gen].apply(get_textstat_metrics)
df_textstat = pd.DataFrame(textstat_metrics.tolist()).add_prefix("TextStat_")

df_out = pd.concat([df_out, df_textstat], axis=1)

print(f"✅ TextStat calculado ({len(df_textstat.columns)} métricas)")

# --------------------------------------------------------
# 🔗 AlignScore o Equivalente Rapido
# --------------------------------------------------------
# -------- SI NO HAY SCORE, CALCULARLO (GPU / batching) --------
print("\n🔗 Calculando AlignScore (versión ligera)...")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"   Usando: {device}")

def compute_pairwise_align(ref_texts, gen_texts, model_name=EMB_MODEL, batch_size=EMB_BATCH, device=device):
    if SentenceTransformer is None:
        raise ImportError("sentence-transformers no está instalado. Instálalo con: pip install sentence-transformers")
    if len(ref_texts) != len(gen_texts):
        raise ValueError("refs y gens deben tener la misma longitud")

    model = SentenceTransformer(model_name, device=device)
    model.max_seq_length = 512

    def encode_list(texts):
        embs = []
        for i in tqdm(range(0, len(texts), batch_size), desc="Encoding", leave=False):
            batch = texts[i:i+batch_size]
            b_emb = model.encode(batch, convert_to_tensor=True, show_progress_bar=False)
            b_emb = torch.nn.functional.normalize(b_emb, p=2, dim=1)
            embs.append(b_emb)
        return torch.cat(embs, dim=0)

    refs_emb = encode_list(ref_texts)
    gens_emb = encode_list(gen_texts)
    refs_emb = refs_emb.to(device)
    gens_emb = gens_emb.to(device)

    with torch.no_grad():
        cosines = torch.sum(refs_emb * gens_emb, dim=1).clamp(-1.0, 1.0).cpu().numpy()
    return cosines

# Calcular y guardar
align_scores = compute_pairwise_align(non_pls, preds)
df_out["AlignScore"] = align_scores

print(f"✅ AlignScore calculado (cosine embedding)")
print(f"   - Media: {sum(align_scores)/len(align_scores):.3f}")

print("\n" + "="*80)
print("✅ TODAS LAS MÉTRICAS CALCULADAS")
print("="*80)

# ============================================================================
# 📊 PASO 2: RESUMEN DE MÉTRICAS - TABLA PROFESIONAL
# ============================================================================

from tabulate import tabulate

print("\n🔧 Preparando tablas de resumen...")

# Renombrar TextStat con nombres cortos
textstat_rename = {
    "TextStat_flesch_reading_ease": "FRE",
    "TextStat_flesch_kincaid_grade": "FKGL",
    "TextStat_smog_index": "SMOG",
    "TextStat_coleman_liau_index": "CLI",
    "TextStat_dale_chall_readability_score": "DCRS",
    "TextStat_gunning_fog": "GFI",
}

# Aplicar renombrado
rename_dict = {k: v for k, v in textstat_rename.items() if k in df_out.columns}
df_out_display = df_out.rename(columns=rename_dict)

# Seleccionar columnas de métricas
metric_columns = [
    "BERT_Precision",
    "BERT_Recall",
    "BERT_F1",
    #"BERT2_F1_Longformer",
    "AlignScore"
] + [v for v in textstat_rename.values() if v in df_out_display.columns]

# Añadir columnas TextStat no renombradas
other_textstat = [col for col in df_out.columns if col.startswith("TextStat_") and col not in textstat_rename]
metric_columns.extend(other_textstat)

# Crear DataFrame de métricas sin duplicados
df_metrics = df_out_display[metric_columns].copy()
df_metrics = df_metrics.loc[:, ~df_metrics.columns.duplicated()]

print(f"✓ Métricas seleccionadas: {len(df_metrics.columns)} columnas")

# ============================================================================
# ✅ CREAR stats_df
# ============================================================================

stats_df = pd.DataFrame({
    'Mean': df_metrics.mean(numeric_only=True),
    'Std': df_metrics.std(numeric_only=True),
    'Min': df_metrics.min(numeric_only=True),
    'Max': df_metrics.max(numeric_only=True),
    'Median': df_metrics.median(numeric_only=True),
}).T

print(f"✓ stats_df creado: {stats_df.shape}")

# ============================================================================
# 📈 TABLA DE MÉTRICAS POR TEXTO (primeros 10)
# ============================================================================

print("\n" + "="*120)
print("📊 MÉTRICAS POR TEXTO (Primeros 10 ejemplos)")
print("="*120)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.3f}'.format)

print("\n")
print(df_metrics.head(10).to_string())
print("\n" + "="*120)

# ============================================================================
# 📊 ESTADÍSTICAS GLOBALES
# ============================================================================

print("\n" + "="*120)
print("📈 ESTADÍSTICAS GLOBALES")
print("="*120 + "\n")

# Preparar datos para tabulate
table_data = []
for stat_name in ['Mean', 'Std', 'Min', 'Max', 'Median']:
    row = [stat_name] + [f"{val:.3f}" for val in stats_df.loc[stat_name]]
    table_data.append(row)

headers = ['Statistic'] + list(stats_df.columns)

print(tabulate(table_data, headers=headers, tablefmt='grid',
               stralign='center', numalign='center'))
print()

# ============================================================================
# 📊 REFERENCIA COMPARATIVA (Modelos del paper CL4Health 2025)
# ============================================================================

reference_data = [
    ["DeepSeek-R1-7B",        16.99, 22.75, 17.69, 12.31, 14.80, 9.45, 0.7955, 0.8496],
    ["Gemini-1.3-Flash",      9.60,  66.87, 8.75,  9.08,  6.90,  5.94, 0.6333, 0.8474],
    ["GPT-4-32K",            12.48,  48.52, 13.39, 11.20, 10.80, 7.41, 0.7801, 0.8519],
    ["GPT-4o",               11.49,  57.13, 11.16,  9.91,  9.09, 6.88, 0.7383, 0.8527],
    ["GPT-3.5-Turbo-16K",    15.52,  28.08, 17.33, 12.59, 13.91, 8.60, 0.8781, 0.8585],
    ["Llama-3.2-3B",         16.42,  21.96, 18.58, 10.79, 15.73, 9.39, 0.8785, 0.8490],
]

ref_headers = ["Model", "CLI ↓", "FRE ↑", "GFI ↓", "SMOG ↓", "FKGL ↓", "DCRS ↓", "AlignScore ↑", "BERTScore ↑"]

print("\n" + "="*120)
print("📊 TABLA DE REFERENCIA (Valores del paper CL4Health-2025)")
print("="*120 + "\n")

print(tabulate(reference_data, headers=ref_headers, tablefmt="grid",
               stralign="center", numalign="center"))
print()

# ============================================================================
# 🎯 RESUMEN EJECUTIVO
# ============================================================================

print("\n" + "="*100)
print("🎯 RESUMEN EJECUTIVO")
print("="*100)

key_metrics = {
    "Semantic Similarity": {
        "BERT F1": df_metrics["BERT_F1"].mean() if "BERT_F1" in df_metrics.columns else None,
        "AlignScore": df_metrics["AlignScore"].mean() if "AlignScore" in df_metrics.columns else None,
    },
    "Readability": {
        "Flesch Ease (FRE)": df_metrics["FRE"].mean() if "FRE" in df_metrics.columns else None,
        "FK Grade Level": df_metrics["FKGL"].mean() if "FKGL" in df_metrics.columns else None,
        "SMOG Index": df_metrics["SMOG"].mean() if "SMOG" in df_metrics.columns else None,
    },
    "Complexity": {
        "Coleman-Liau": df_metrics["CLI"].mean() if "CLI" in df_metrics.columns else None,
        "Dale-Chall": df_metrics["DCRS"].mean() if "DCRS" in df_metrics.columns else None,
        "Gunning Fog": df_metrics["GFI"].mean() if "GFI" in df_metrics.columns else None,
    }
}

for category, metrics in key_metrics.items():
    print(f"\n📌 {category}:")
    for metric_name, value in metrics.items():
        if value is not None:
            # Emoji según valor
            if "F1" in metric_name or "AlignScore" in metric_name:
                emoji = "✅" if value >= 0.7 else "⚠️" if value >= 0.5 else "❌"
            elif "Flesch" in metric_name or "FRE" in metric_name:
                emoji = "✅" if value >= 60 else "⚠️" if value >= 50 else "❌"
            elif any(x in metric_name for x in ["Grade", "SMOG", "Coleman", "Dale", "Gunning"]):
                emoji = "✅" if value <= 10 else "⚠️" if value <= 12 else "❌"
            else:
                emoji = "📊"

            print(f"   {emoji} {metric_name:.<35} {value:>8.3f}")

print("="*100)

# =====================================================================
# 🎯 RESUMEN EJECUTIVO (con comparación a Llama-3.2-3B)
# =====================================================================

print("\n" + "="*100)
print("🎯 RESUMEN EJECUTIVO (comparado con Llama-3.2-3B)")
print("="*100)

# === 1️⃣ Valores de referencia (Llama-3.2-3B) ===
llama3_ref = {
    "CLI": 16.42,
    "FRE": 21.96,
    "GFI": 18.58,
    "SMOG": 10.79,
    "FKGL": 15.73,
    "DCRS": 9.39,
    "AlignScore": 0.8785,
    "BERT_F1": 0.849
}

# === 2️⃣ Tu resumen de métricas ===
key_metrics = {
    "Semantic Similarity": {
        "BERT_F1": df_metrics["BERT_F1"].mean() if "BERT_F1" in df_metrics.columns else None,
        "AlignScore": df_metrics["AlignScore"].mean() if "AlignScore" in df_metrics.columns else None,
    },
    "Readability": {
        "FRE": df_metrics["FRE"].mean() if "FRE" in df_metrics.columns else None,
        "FKGL": df_metrics["FKGL"].mean() if "FKGL" in df_metrics.columns else None,
        "SMOG": df_metrics["SMOG"].mean() if "SMOG" in df_metrics.columns else None,
    },
    "Complexity": {
        "CLI": df_metrics["CLI"].mean() if "CLI" in df_metrics.columns else None,
        "DCRS": df_metrics["DCRS"].mean() if "DCRS" in df_metrics.columns else None,
        "GFI": df_metrics["GFI"].mean() if "GFI" in df_metrics.columns else None,
    }
}

# === 3️⃣ Mostrar métricas comparadas ===
for category, metrics in key_metrics.items():
    print(f"\n📌 {category}:")
    for metric_name, value in metrics.items():
        if value is not None:
            # Valor de referencia (si existe)
            ref_val = llama3_ref.get(metric_name, None)

            # Emoji según desempeño relativo
            if "F1" in metric_name or "AlignScore" in metric_name:
                emoji = "✅" if value >= 0.7 else "⚠️" if value >= 0.5 else "❌"
            elif "FRE" in metric_name:
                emoji = "✅" if value >= 60 else "⚠️" if value >= 50 else "❌"
            elif any(x in metric_name for x in ["Grade", "SMOG", "CLI", "DCRS", "GFI", "FKGL"]):
                emoji = "✅" if value <= 10 else "⚠️" if value <= 12 else "❌"
            else:
                emoji = "📊"

            # Mostrar línea con comparación
            if ref_val is not None:
                diff = value - ref_val
                arrow = "⬆️" if diff > 0 else "⬇️" if diff < 0 else "➡️"
                print(f"   {emoji} {metric_name:<20} {value:>8.3f}  ({arrow} vs Llama3 {ref_val:>8.3f})")
            else:
                print(f"   {emoji} {metric_name:<20} {value:>8.3f}")

print("="*100)
# ============================================================================
# 💾 EXPORTAR A CSV
# ============================================================================

df_metrics.to_csv("metrics_complete.csv", index=False)
stats_df.T.to_csv("metrics_summary.csv")

print("\n💾 Tablas guardadas:")
print("   - metrics_complete.csv (todas las métricas por texto)")
print("   - metrics_summary.csv (estadísticas resumidas)")
print()

📊 CALCULANDO MÉTRICAS

✓ Columna de referencia: response
✓ Columna generada: generated_text
✓ Total textos: 50

📊 Calculando BERTScore (modelo base)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ AlignScore calculado (cosine embedding)
   - Media: 0.872

✅ TODAS LAS MÉTRICAS CALCULADAS

🔧 Preparando tablas de resumen...
✓ Métricas seleccionadas: 13 columnas
✓ stats_df creado: (5, 13)

📊 MÉTRICAS POR TEXTO (Primeros 10 ejemplos)


   BERT_Precision  BERT_Recall  BERT_F1  AlignScore    FRE   FKGL   SMOG    CLI   DCRS    GFI  TextStat_automated_readability_index  TextStat_difficult_words  TextStat_linsear_write_formula
0           0.870        0.840    0.855       0.922 47.835 10.524 12.636 11.867 10.861 12.940                                11.415                        46                          10.714
1           0.874        0.840    0.857       0.917 46.530 10.553 13.274 12.692 12.048 13.567                                12.067                        58                          10.429
2           0.853        0.832    0.842       0.858 44.821 10.791 13.274 12.721 11.569 13.769                                11.889                        55                          10.286


In [ ]:
import textwrap
import random

# =====================================================================
# EJEMPLO DE SALIDA SIMPLIFICADA
# =====================================================================
print("\n🧾 Ejemplo de salida simplificada:\n" + "="*50)

# Seleccionamos un ejemplo aleatorio del DataFrame
idx = random.randint(0, len(df_out) - 1)

print(f"\n🔹 Índice: {idx}")

# Función auxiliar para mostrar texto con ancho fijo y saltos de línea
def mostrar_texto_formateado(texto, ancho=150):
    # Reemplaza saltos de línea escapados por saltos reales, si los hay
    texto = texto.replace("\\n", "\n")
    # Divide el texto en líneas y aplica textwrap a cada una
    lineas = []
    for linea in texto.splitlines():
        # Ajusta cada línea a máximo 'ancho' caracteres
        lineas.extend(textwrap.wrap(linea, width=ancho, replace_whitespace=False) or [""])
    print("\n".join(lineas))

# Texto original (hasta 1000 caracteres)
print("\n--- Texto original (source) ---")
mostrar_texto_formateado(df_out.loc[idx, "source"][:1000], ancho=150)

# Texto simplificado generado
print("\n--- Texto simplificado generado ---")
mostrar_texto_formateado(df_out.loc[idx, "generated_text"], ancho=150)

print("\n" + "="*50)



🧾 Ejemplo de salida simplificada:

🔹 Índice: 46

--- Texto original (source) ---
Four placebo‐controlled RCTs including 206 participants were included. Three trials included an 8 to 16 week induction phase with tapering
corticosteroids (prednisone, prednisolone or methylprednisolone) as induction therapy. Anti‐tuberculous therapy included monotherapy with clofazimine,
combination therapy with clofazimine, rifampin, ethambutol, and dapsone or combination therapy with clarithromycin, rifabutin and clofazimine. All of
the studies were rated as unclear risk of bias for allocation concealment, three were rated as unclear risk of bias for random sequence generation and
two were rated as unclear risk of bias for blinding or participants and personnel. There was a statistically significant difference in relapse rates
favoring anti‐tuberculous therapy over placebo. Thirty‐nine per cent (44/112) of patients in the anti‐tuberculous therapy group relapsed at 9 months
to 2 years compared to 67% (6

In [ ]:

# Filtrar los casos con AlignScore < 0.3
low_align = df_out[df_out["AlignScore"] < 0.3]

# Mostrar cuántos hay
print(f"🧮 {len(low_align)} ejemplos con AlignScore < 0.3\n")

# Iterar sobre los primeros 10 ejemplos
for i, row in enumerate(low_align[["source", "response", "generated_text", "AlignScore"]].head(10).itertuples(), start=1):
    print(f"\n🔹 Ejemplo {i} | AlignScore: {row.AlignScore:.3f}")
    print("=" * 150)

    # Texto original (source)
    print("\n--- Texto original (source) ---")
    print(textwrap.fill(row.source.replace("\\n", "\n"), width=150, replace_whitespace=False))

    # Respuesta de referencia (ground truth)
    print("\n--- Resumen de referencia (response) ---")
    print(textwrap.fill(row.response.replace("\\n", "\n"), width=150, replace_whitespace=False))

    # Texto generado por el modelo
    print("\n--- Texto simplificado generado ---")
    print(textwrap.fill(row.generated_text.replace("\\n", "\n"), width=150, replace_whitespace=False))

    print("-" * 150)


🧮 0 ejemplos con AlignScore < 0.3



In [ ]:
# Filtrar los casos con AlignScore > 0.8
low_align = df_out[df_out["AlignScore"] > 0.8]

# Mostrar cuántos hay
print(f"🧮 {len(low_align)} ejemplos con AlignScore > 0.8\n")

# Iterar sobre los primeros 10 ejemplos
for i, row in enumerate(low_align[["source", "response", "generated_text", "AlignScore"]].head(10).itertuples(), start=1):
    print(f"\n🔹 Ejemplo {i} | AlignScore: {row.AlignScore:.3f}")
    print("=" * 150)

    # Texto original (source)
    print("\n--- Texto original (source) ---")
    print(textwrap.fill(row.source.replace("\\n", "\n"), width=150, replace_whitespace=False))

    # Respuesta de referencia (ground truth)
    print("\n--- Resumen de referencia (response) ---")
    print(textwrap.fill(row.response.replace("\\n", "\n"), width=150, replace_whitespace=False))

    # Texto generado por el modelo
    print("\n--- Texto simplificado generado ---")
    print(textwrap.fill(row.generated_text.replace("\\n", "\n"), width=150, replace_whitespace=False))

    print("-" * 150)


🧮 43 ejemplos con AlignScore > 0.8


🔹 Ejemplo 1 | AlignScore: 0.922

--- Texto original (source) ---
Background
This updated Cochrane Review of reminiscence therapy (RT) for dementia was first published in 1998, and last updated in 2005. RT involves
the discussion of memories and past experiences with other people using tangible prompts such as photographs or music to evoke memories and stimulate
conversation. RT is implemented widely in a range of settings using a variety of formats. 
Objectives
To assess the effects of RT on people living
with dementia and their carers, taking into account differences in its implementation, including setting (care home, community) and modality (group,
individual). 
Search methods
We searched ALOIS (the Cochrane Dementia and Cognitive Improvement Group's Specialized Register) on 6 April 2017 using
the search term 'reminiscence.' 
Selection criteria
We included all randomised controlled trials of RT for dementia in which the duration of the
interventi

Inspección de la calidad de 10

In [ ]:
import textwrap

for i, text in enumerate(df_out["generated_text"].head(10), start=1):
    print(f"\n🔹 Ejemplo {i}:")
    print(textwrap.fill(text, width=150))  # wrap de 150 caracteres por línea
    print("-" * 150)


🔹 Ejemplo 1:
**Plain Title:** Does Tonsillectomy Help Children with Periodic Fever, Aphthous Stomatitis, Pharyngitis and Cervical Adenitis Syndrome? **Rationale:**
Periodic fever, aphthous stomatitis, pharyngitis and cervical adenitis (PFAPA) syndrome is a rare condition that causes recurring episodes of fever,
mouth sores, sore throat, and swollen lymph nodes in the neck. Some people think that removing the tonsils might help alleviate these symptoms. But
there's still much we don't know about this condition. The reason for this study is to see if tonsillectomy (removing the tonsils) helps children with
PFAPA syndrome feel better. We want to compare how many kids who had their tonsils removed did better than those who didn't. We're looking for answers
like: "Do kids who get their tonsils removed have fewer episodes of fever?" or "Do they need to take medicine more often?" **Trial Design:** This
study looked at how effective tonsillectomy was in treating children with PFAPA syndrome. 